<a href="https://colab.research.google.com/github/sisifo3/P_T_3/blob/main/Densenet121_CheXpert_NIH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_001.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_002.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_003.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_004.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_005.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_006.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_007.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_008.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_009.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_010.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_011.tar.gz
#!tar -xf /content/drive/MyDrive/NIH_Dataset_total/images_012.tar.gz

In [ ]:
!unzip /content/drive/MyDrive/NIH_dataset/CheXpert-v1.0-small-1.zip 

In [2]:
import cv2
import numpy as np
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates
random_state = np.random.RandomState(0)

def clahe(image_name):
    image = cv2.imread(image_name, 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    he_img = clahe.apply(image)
    image = cv2.cvtColor(he_img, cv2.COLOR_GRAY2RGB)
    return Image.fromarray(image)


In [3]:
from torchvision import transforms
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


'''
cropSize = 320

#for NIh dataset
transform = transforms.Compose([
                    #transforms.RandomResizedCrop(cropSize),
                    transforms.Resize((320, 320)),
                    transforms.RandomHorizontalFlip(),
                    transforms.RandomRotation(10),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                    ])
'''

In [3]:
from torchvision import transforms
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#for CheXpert
transform = transforms.Compose([
                                transforms.Resize((320, 320)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                ])

In [4]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset

def getImagesLabels(filename):

    df = pd.read_csv(filename)
    relevant_cols = ['path','No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis','Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices','Hernia','Mass',
                    'Fibrosis','Infiltration','Nodule','Emphysema','Pleural_Thickening']


    df = df[relevant_cols]
    df = df.replace(np.nan, 0.0)

    df = df.replace(-1.0, 0.0)


    X = df['path']
    y = df[['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis','Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices','Hernia','Mass',
                    'Fibrosis','Infiltration','Nodule','Emphysema','Pleural_Thickening']]

    return np.asarray(X), np.asarray(y)



#filename = '/content/drive/MyDrive/NIH_Dataset_total/final_cheXpert_NIH_train.csv'
#x1, y1 = getImagesLabels(filename)

In [5]:
import os
import numpy as np


def getExistValues(train_images, train_labels):
  val2 = ['done first']
  val2 = np.array(val2)
  val3 = [0]*21
  val3= [float(item) for item in val3]
  val3 = np.array(val3)

  i = 0

  for val in train_images:
    path = val
    isExist = os.path.exists(path)
    if isExist == True:
      val2 = np.append(val2, val)
      val3 = np.vstack([val3, train_labels[i]])
    i = i + 1


  val2 = np.delete(val2,0)
  val3 = np.delete(val3, 0, axis=0)

  return  np.asarray(val2), np.asarray(val3)



In [6]:
class CheXpertDataset(Dataset):
    def __init__(self, image_list, labels, transform=None, test=False):
        self.image_names = image_list
        self.labels = labels
        self.transform = transform
        self.test = test

    def __getitem__(self, index):
        image_name = self.image_names[index]
        image = clahe(image_name)
        label = self.labels[index]
        image = self.transform(image)
        return image, torch.FloatTensor(label)

    def __len__(self):
       
        return len(self.image_names)

In [7]:
import torch
import  torchvision
import torch.nn as nn
from torchsummary import summary

class DenseNet121(nn.Module):

    def __init__(self, out_size, test=False):
        super(DenseNet121, self).__init__()
        self.test = test
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, num_ftrs),
            nn.BatchNorm1d(num_ftrs),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_ftrs//2),
            nn.BatchNorm1d(num_ftrs//2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(num_ftrs//2, out_size),
        )

    def forward(self, x):
        x = self.densenet121(x)

        if self.test:
            return torch.sigmoid(x)
        else:
            return x


In [8]:
import torch
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import os

class Trainer():
    def __init__(self, model, train_loader, val_loader):
        self.model = model
        self.train_loader = train_loader
        self.val_loader = val_loader

        self.losses = {'train':[], 'validation':[]}


    def compile(self, lr, loss_fn, scheduler=False):
        
        self.learning_rate = lr
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=1e-5)
        self.loss_fn = loss_fn

        if (scheduler):
            self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.1, patience=2, verbose=True, min_lr=1e-6)
  
    def load_checkpoint(self, log_path, model_path):
        """ Function to load checkpoints from a previously saved model
        Arguments:
            log_path : path to saved log file
            model_path : path to saved model
        Returns:
            start_epoch : the epoch from which to resume the training
            model : the model with loaded weights
            optimizer : the optimizer with the previous state
            scheduler : the scheduler with the previous state
            losses : the dictionary populated with loss values from previous epochs
        """
        # Note: Input model & optimizer should be pre-defined. This routine only updates their states
        start_epoch = 0

        self.log_path = log_path
        self.model_path = model_path

        print(self.log("\nLoading checkpoint from '{}'\n".format(model_path)))

        checkpoint = torch.load(model_path)
        start_epoch = checkpoint['epoch'] + 1
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        try:
            self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        except:
            pass

        self.losses = checkpoint['loss']
        val_min = min(self.losses['validation'])

        print(self.log("--> Loaded checkpoint from '{}'\nResuming training from epoch {}\n\n"
                    .format(model_path, start_epoch)))

        return start_epoch, self.model, self.optimizer, self.scheduler, self.losses, val_min
        # return start_epoch, self.model, self.optimizer, self.losses, val_min


    def epoch_train(self, print_every=50):
        model = self.model
        loss_fn = self.loss_fn
        optimizer = self.optimizer
        #device = self.device
        print("trainin 2")
        train_loss = 0.0
        model.train()
        for batch_idx, (data, target) in enumerate(self.train_loader):
            #print(target)
            #print(data)  
            data, target = data.float().to(device), target.float().to(device)

            optimizer.zero_grad()

            pred = model(data)
            loss = loss_fn(pred, target)

            train_loss += ((1 / (batch_idx + 1)) * (loss.item()/data.size(0) - train_loss))
            loss.backward()
            optimizer.step()

            if (batch_idx % print_every == 0):
                print('Epoch {}\tBatch [{}/{}]\t\tTraining Loss: {}'.format(self.epoch+1, batch_idx+1, len(self.train_loader), train_loss))

        return train_loss

    def epoch_val(self):
        model = self.model
        loss_fn = self.loss_fn
        #device = self.device

        valid_loss = 0.0
        model.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(self.val_loader):
                data, target = data.float().to(device), target.float().to(device)

                val_pred = model(data)
                val_loss = loss_fn(val_pred, target)

                valid_loss += ((1 / (batch_idx + 1)) * (val_loss.item()/data.size(0) - valid_loss))

        return valid_loss

    def train(self, n_epochs, batch_size, log_path=None, model_path=None):
  
        start = datetime.now()

        model = self.model
        optimizer = self.optimizer
        try:
            scheduler = self.scheduler
        except:
            pass
        losses = self.losses
        valid_loss_min = np.Inf
        start_epoch = 0

        if (log_path is None and model_path is None):
            self.log_path = str(start.strftime('%d-%m-%Y-%H:%M:%S')+'_train_log')
            self.model_path = '{}_model.pt'.format(start.strftime('%d-%m-%Y-%H:%M:%S'))
            self.log('Learning rate: {}, Batch size: {}\n\n'.format(self.learning_rate, batch_size))

        elif (os.path.exists(model_path) and os.path.exists(log_path)):
            # start_epoch, model, optimizer, scheduler, losses = self.load_checkpoint(log_path, model_path)
            start_epoch, model, optimizer, scheduler, losses, valid_loss_min = self.load_checkpoint(log_path, model_path)
            valid_loss_min = min(losses['validation'])

        else:
            print('[!] Specified model path or log path does not exist.')
            return

        # loss_fn = self.loss_fn

        

        checkpoint = {}

        for self.epoch in range(start_epoch, start_epoch+n_epochs):

            #print("we start trinning")
            train_loss = self.epoch_train(print_every=500)

            valid_loss = self.epoch_val()

            print(self.log('\nEpoch: [{}/{}] \tTraining Loss: {:.5f} \tValidation Loss: {:.5f}\n'.format(
                                                    self.epoch+1, start_epoch+n_epochs, train_loss, valid_loss)))
            print('-'*100)

                 #####----CHECKPOINTING----#####
            if (valid_loss < valid_loss_min):
                print(self.log("Saving model.  Validation loss:... {:.5f} --> {:.5f}\n".format(valid_loss_min, valid_loss)))
                print('*'*100)
                valid_loss_min = valid_loss

                checkpoint['model_state_dict'] = model.state_dict()
                checkpoint['optimizer_state_dict'] = optimizer.state_dict()
                try:
                    checkpoint['scheduler_state_dict'] = scheduler.state_dict()
                except:
                    pass
                print()

            try:
                scheduler.step(valid_loss)
            except:
                pass

            losses['train'].append(train_loss)
            losses['validation'].append(valid_loss)

            checkpoint['epoch'] = self.epoch
            checkpoint['loss'] = losses
            torch.save(checkpoint, self.model_path)

            self.draw_loss_curve('{}_losses.png'.format(self.model_path.split('_')[0]))

        end = datetime.now()
        time = str(end - start).split('.')[0]
        print(self.log("\nCompleted training in {}\n".format(time)))

        return model, losses

    def log(self, info):
        """ Function to create and update the log file
        Arguments:
            info : the update information to write on the log file
        Returns:
            info : the logged information to be printed while training
        """
        log_path = self.log_path

        if not os.path.exists(log_path):
            file = open(log_path, 'w')
            file.write(info)
            file.close()
        else:
            file = open(log_path, 'a')
            file.write(info)
            file.close()

        return info.strip('\n')

    def draw_loss_curve(self, fpath, losses=None):
        """ Function to generate loss curve for the training process
        Arguments:
            fpath : the filepath to save the loss curve in
        """
        if losses is None:
            losses = self.losses
        # plt.ylim([0,2])
        plt.plot(losses['train'], label='Training loss')
        plt.plot(losses['validation'], label='Validation loss')
        plt.legend()
        plt.savefig(fpath)
        # plt.show()
        plt.close()  

In [ ]:
import sys
import numpy as np
import torch
import torch.nn as nn
import math

from torch.utils.data import DataLoader


train_csv = '/content/drive/MyDrive/NIH_Dataset_total/final_Only_cheXpert_train.csv'
#train_csv = '/content/drive/MyDrive/NIH_Dataset_total/final_Only_NIH_train.csv'

train_images, train_labels = getImagesLabels(train_csv)

##########################################3
#train_images, train_labels = getExistValues(train_images, train_labels)

l = len(train_images)
n = math.floor(l - (l * .2))
print(n)
#########################

#train = 268000
#val = 67000

val_images, val_labels = train_images[n+1:], train_labels[n+1:]
train_images, train_labels = train_images[:n], train_labels[:n]

print('No. of images:\n\t|--Training: {}\n\t|--Validation: {}\n'.format(len(train_images), len(val_images)))

train_dataset = CheXpertDataset(train_images, train_labels, transform)
val_dataset = CheXpertDataset(val_images, val_labels, transform)

### hyperparameters ###
batchSize = 4
lr = 0.0001
epochs = 1
num_classes = 21

### data loaders ###
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True,  num_workers=2, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False,  num_workers=2, pin_memory=True)

### initializing the model ###
model = DenseNet121(num_classes)
model = torch.nn.DataParallel(model)
#model.cuda()

### training ###
trainer = Trainer(model, train_loader, val_loader)

# moving the model to device
model.to(device)


pos_weights = [3.059119740473539, 28.90637155028709, 9.992811313782795, 2.1230429717468104, 35.34269540605269, 4.909329226933583,
14.824158864080667, 41.38345159994645, 5.714152140519602, 12.425494079133493, 2.2535681260804887, 93.43059892137383,
36.02323008849557, 1.8814665390815597, 1476.2334801762115, 57.076203671631454, 198.2465834818776, 15.867806841046278,
52.084058888712995, 132.598406374502, 98.15198107628622]

pos_weights = torch.FloatTensor(pos_weights).cuda()
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weights)

trainer.compile(lr, criterion, scheduler=True)

trainer.train(epochs, batchSize)

178730
No. of images:
	|--Training: 178730
	|--Validation: 44682



Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth


  0%|          | 0.00/30.8M [00:00<?, ?B/s]

trainin 2
Epoch 1	Batch [1/44683]		Training Loss: 0.5720852613449097
Epoch 1	Batch [501/44683]		Training Loss: 0.31706425607085453


In [ ]:
import torch
torch.cuda.empty_cache()